In [10]:
import numpy as np
import math
import matplotlib.pyplot as plt
import time
import scipy as sp
import seaborn as sns
import pandas as pd




betPU, alfPU       =  1.3, -0.9
betK, alfK         =  0.8,  0.6
gamPU, gamK        =  (1 + alfPU**2)/betPU, (1 + alfK**2)/betK
muPUK, muKPU       =  1.51, 2*np.pi*1.34 - 1.51
total_angle        =  2*np.pi*1.34
etaPUK, etaKPU     =  -0.01, -0.01     #eta PUK must be reletevely small, so they dont change slices so fast... 0.02 breaks the system ... It is the limit case!!!! very sensitive on etaPUK --> 2 much unwanted mixing, also etaKPU cant go very low(limit case 0.3) because then the mixing is bad
sigdelt            = .0001            # rms momentum spread in unit
Th, Trev           =  .1e-6, 5.0e-6, # half width of simulation window and revolution period in s
Tovlap, NSamp, add = 4*sigdelt*abs(etaPUK + etaKPU)*Trev, 800, '2000'
    #gain               =  0.00018  #as long as I reduce the eta, i have to reduce gains as well... see book
    #gainL              =  3e-8
over_sample_factor=10
DTSamp   = Th/NSamp
NSampext = math.ceil( Tovlap/DTSamp )
inds = np.arange( (-NSampext)*over_sample_factor , (NSamp+1+NSampext + 1)*over_sample_factor )
t_bins = inds*DTSamp/over_sample_factor -Th/2
wlL, wrL = -1.6e-9, 1.6e-9
wlH, wrH = -1.6e-9, 1.6e-9

rng = np.random.default_rng(2275405 )

#time delay cooling here!~!!!!!1
#this approach -> bin centers main = only bin centers main
#also cut the time offset ->Th+Dtsamp+Tovlap

#datin  = time.time()  # record start time - to estimate run duration at end

def running_average(x, y, bins):
     """
     Compute the running average of y over fixed intervals in x efficiently using NumPy.

    Parameters:
    x (array-like): Array of x-values (sorted or unsorted).
    y (array-like): Array of y-values corresponding to x.
    interval_width (float): The width of the intervals.

    Returns:
    bin_centers (numpy.ndarray): The centers of the bins.
    averages (numpy.ndarray): The average y-values for each bin.
     """
    # Ensure inputs are NumPy arrays
     x = np.array(x)
     y = np.array(y)

    # Bin the data using NumPy digitize
     bin_indices = np.digitize(x, bins) - 1  # Get zero-based bin indices

    # keep only elements within bins
     valid_input_indices = (bin_indices > 0) & (bin_indices < len(bins) - 1)

    # Preallocate array for averages
     averages = np.zeros(len(bins) - 1, dtype=float)
    
    # Use NumPy's bincount for fast bin-based summation and counts
     bin_sums   = np.bincount(bin_indices[valid_input_indices], minlength=len(bins) - 1, weights=y[valid_input_indices])
     bin_counts = np.bincount(bin_indices[valid_input_indices], minlength=len(bins) - 1)
    
    # Avoid division by zero: compute averages where bin_counts > 0
     non_empty_bins = bin_counts > 0
     averages[non_empty_bins] = bin_sums[non_empty_bins] / bin_counts[non_empty_bins]

    # Compute bin centers
     bin_centers = (bins[:-1] + bins[1:]) / 2

     return bin_centers, averages, bin_counts, bin_sums


    

# Routine to do do the exchange of .. ended up by regrouping from scratch 
def PartsExchange( ptsEarly, ptsLate,Th ):
     ptsEarlyNew, ptsLateNew = [], []
     for x, xp, delt, Tau in ptsEarly:
        if Tau < Th/2:  # should remain in "early group
            ptsEarlyNew += [ [x, xp, delt, Tau] ]
        else:
            ptsLateNew += [ [x, xp, delt, Tau - Th] ]
     for x, xp, delt, Tau in ptsLate:
        if Tau > -Th/2:
            ptsLateNew += [ [x, xp, delt, Tau] ]
        else:
            ptsEarlyNew += [ [x, xp, delt, Tau + Th] ]
     return ptsEarlyNew, ptsLateNew


def respH( Dt ):
   # if Dt < 1.*wlH or Dt > 1.*wrH:
    #    print( f' ===> function respH called with Dt ={1e6*Dt:8.4f}' )
      return (1 - 27e18*Dt**2)*(1 - 3.e18*Dt**2)*(1 - 1.15e18*Dt**2)*(1 - (.625e9*Dt)**2)**7
#   return (1 - 23e16*Dt**2)*(1 - 3.e16*Dt**2)*(1 - 1.15e16*Dt**2)*(1 - (.625e8*Dt)**2)**7
#   return (1 - 25e-12*Dt^2)*(1 - 3e-12*Dt^2)*(1 - 1.15e-12*Dt^2)*(1 - (Dt/1.5)^2)^6


def respL( Dt ):
   # if Dt < 1.*wlL or Dt > 1.*wrL:
    #    print( f' ===> function respL called with Dt ={1e6*Dt:8.4f}' )
      return  (63e9*Dt)*(1 - 6.5e18*Dt**2)*(1 - 1.55e18*Dt**2)*(1 - (.625e9*Dt)**2)**8

def TimeTraces(ptsEarly, ptsMain, ptsLate, bins,Th):
     """
    Generate time traces without applying the `Th / 2` offset internally.
     """
    
     ptsEarly = np.array(ptsEarly)
     ptsMain = np.array(ptsMain)
     ptsLate = np.array(ptsLate)
     # Compute histograms of beam distribution
     bin_centers_early, averages_early, count_early, sum_early = running_average(
        ptsEarly[:, 3]-Th, ptsEarly[:, 0], bins
      )
     bin_centers_main, averages_main, count_main, sum_main = running_average(
        ptsMain[:, 3], ptsMain[:, 0], bins
      )
     bin_centers_late, averages_late, count_late, sum_late = running_average(
        ptsLate[:, 3]+Th, ptsLate[:, 0], bins
      )

    # Compute time traces
     TraceH_early = np.convolve(respH_new_once, sum_early, 'same')
     TraceH_main = np.convolve(respH_new_once, sum_main, 'same')
     TraceH_late = np.convolve(respH_new_once, sum_late, 'same')
    
     TraceL_early = np.convolve(respL_new_once, count_early, 'same')
     TraceL_main = np.convolve(respL_new_once, count_main, 'same')
     TraceL_late = np.convolve(respL_new_once, count_late, 'same')

     TraceH = TraceH_early + TraceH_main + TraceH_late
     TraceL = TraceL_early + TraceL_main + TraceL_late #i am multiplying with this factor in order to amplify the signal!!!!!
    
    
    
     return bin_centers_main, TraceH, TraceL
def kickH(pts, signalH,gain):
      corrected_pts = []
      for x, xp, delt, Tau in pts:
        # Interpolate signalH at the particle's Tau
        signal_value = signalH(Tau)
        # Correct xp using the signal value (define your correction logic)
        corrected_xp = xp +gain * signal_value  # I add the signal!!!!!
        # Append the updated particle to the new list
        corrected_pts.append([x, corrected_xp, delt, Tau])
      return corrected_pts

def kickL(pts, signalL,gainL):
     corrected_pts = []
     for x, xp, delt, Tau in pts:
        # Interpolate signalL at the particle's Tau
        signal_value = signalL(Tau)
        # Correct xp using the signal value (define your correction logic)
        corrected_delt = delt + gainL * signal_value  # Example: subtracting scaled signal
        # Append the upd1.0pend([x, xp, corrected_delt, Tau])
        corrected_pts.append([x, xp, corrected_delt, Tau])
     return corrected_pts

respH_new_once = respH(np.arange(wlH, wrH, DTSamp/over_sample_factor ))
respL_new_once = respL(np.arange(wlL, wrL, DTSamp/over_sample_factor ))

def Cooling(gain,gainL,Nparts,Nturns):
    
  



   
# definition of the transverse "response function", which is given by the 
#    routine resp for the interval -wlH < Dt < wrH and 0 outside
    
#rng = np.random.default_rng(18022005)
    #rng = np.random.default_rng(2275405 )   #this gives us the random seed!!!!

# Total width (particles of both colors) of beam time window is 2*Th ('h' for half)
#  for sampling each sub-window (red or blue) divided into NSamp intervals. This
#  means samp,ing rate DTsamp and NSamp + 1 points. On both ends to be added 
#  ceiling( Tovlap/DTSamp ) points   
    

    print('========>', NSamp, NSampext, DTSamp, Th, (wrH - wlH)/DTSamp)

    
# Routine to generate time traces

   

   

    
# Computation of transverse transfer matrix elements
    projPU11 =  betPU**(1/2)
    projPU21 = -alfPU/betPU**(1/2)
    projPU22 =  1/betPU**(1/2)

    matPUK11 = ((betK/betPU)**(1/2))*(np.cos(muPUK) + alfPU*np.sin(muPUK))
    matPUK12 = ((betPU*betK)**(1/2))*np.sin(muPUK)
    matPUK21 = ((alfPU-alfK)*np.cos(muPUK) - (1+alfPU*alfK)*np.sin(muPUK))/((betPU*betK)**(1/2))
    matPUK22 = ((betPU/betK)**(1/2))*(np.cos(muPUK) - alfK*np.sin(muPUK))

    matKPU11 = ((betPU/betK)**(1/2))*(np.cos(muKPU) + alfK*np.sin(muKPU))
    matKPU12 = ((betPU*betK)**(1/2))*np.sin(muKPU)
    matKPU21 = ((alfK-alfPU)*np.cos(muKPU) - (1+alfPU*alfK)*np.sin(muKPU))/((betPU*betK)**(1/2))
    matKPU22 = ((betK/betPU)**(1/2))*(np.cos(muKPU) - alfPU*np.sin(muKPU))

# Generate macroparticles in two groups with red ones and blue ones
#    .. say we start with red particles the early ones
    ptsRed  = []
    ptsBlue = []
    for _ in range(Nparts):
      Tau = rng.uniform(-Th, Th)
      xn, xpn, deltn = rng.normal(0., 1.), rng.normal(0., 1.), rng.normal(0., 1.)
      if Tau < 0: 
        ptsRed += [ [projPU11*xn, projPU21*xn + projPU22*xpn, 
                     deltn*sigdelt, Tau + Th/2] ]
      else:
        ptsBlue += [ [projPU11*xn, projPU21*xn + projPU22*xpn, 
                      deltn*sigdelt, Tau - Th/2] ]


    emittance=np.zeros(Nturns+2)
    geo_emittance=np.zeros(Nturns+2)
    rms_momentum_spread=np.zeros(Nturns+2)
    beam=[]
    xp_beam=[]
    x_beam=[]
    beam=ptsRed+ptsBlue
    beam = np.array(beam)
    x_beam=beam[:,0]
    xp_beam=beam[:,1]
    x_initial=x_beam
    xp_initial=xp_beam
    tau_initial=beam[:,3]
    delt_initial=beam[:,2]
    counts_initial, bins_initial = np.histogram(delt_initial, bins=30)
    emittance[0]=np.sqrt(np.mean(x_initial**2)*np.mean(xp_initial**2)-np.mean(x_initial*xp_initial)**2)   
    geo_emittance[0]=((gamPU*np.mean(x_initial**2)+2*alfPU*np.mean(x_initial*xp_initial)+betPU*np.mean(xp_initial**2)))/2
#emittance_individual_particles[0][:]=(gamPU*x_beam[:10]**2+2*alfPU*(x_beam[:10]*xp_beam[:10])+betPU*(xp_beam[:10]**2))/2
    rms_momentum_spread[0]=np.sqrt(np.mean(delt_initial**2)-np.mean(delt_initial)**2)



    ptsRedprev = ptsRed
    ptsRed = [ [matPUK11*x + matPUK12*xp, matPUK21*x + matPUK22*xp, delt, 
               Tau + etaPUK*delt*Trev] for x, xp, delt, Tau in ptsRed ]  # no action at kicker
    ptsRed = [ [matKPU11*x + matKPU12*xp, matKPU21*x + matKPU22*xp, delt, 
               Tau + etaKPU*delt*Trev] for x, xp, delt, Tau in ptsRed]

# Prepare for tracking of blue particles and track them once around
    ptsBlue, ptsRed = PartsExchange( ptsBlue, ptsRed,Th )
    ptsBlueprev = ptsBlue
    time2,TrHBlue, TrLBlue = TimeTraces( ptsRedprev, ptsBlue, ptsRed,t_bins,Th )

    ptsBlue = [ [matPUK11*x + matPUK12*xp, matPUK21*x + matPUK22*xp, delt, 
            Tau + etaPUK*delt*Trev] for x, xp, delt, Tau in ptsBlue ]  # no action at kicker
#ptsBlue=kickH(ptsBlue,cs)
    ptsBlue = [ [matKPU11*x + matKPU12*xp, matKPU21*x + matKPU22*xp, delt, 
            Tau + etaKPU*delt*Trev] for x, xp, delt, Tau in ptsBlue]
    TrHBlue_prev, TrLBlue_prev = TrHBlue, TrLBlue
# Prepare for 2nd tracking of red particles and track them once around
    ptsRed, ptsBlue = PartsExchange( ptsRed, ptsBlue,Th )
    ptsRedprev = ptsRed
    time2,TrHRed, TrLRed = TimeTraces( ptsBlueprev, ptsRed, ptsBlue,t_bins,Th )
#cs = sp.interpolate.CubicSpline( DTSamp*np.arange(-NSampext, NSamp + 1 + NSampext)-Th/2, 
 #                 TrHRed[-NSampext:] + TrHRed[:NSamp + 1 + NSampext] )
    ptsRed = [ [matPUK11*x + matPUK12*xp, matPUK21*x + matPUK22*xp, delt, 
            Tau + etaPUK*delt*Trev] for x, xp, delt, Tau in ptsRed ]  # no action at kicker
#ptsRed=kickH(ptsRed,cs)
    ptsRed = [ [matKPU11*x + matKPU12*xp, matKPU21*x + matKPU22*xp, delt, 
            Tau + etaKPU*delt*Trev] for x, xp, delt, Tau in ptsRed]
    TrHRed_prev, TrLRed_prev = TrHRed, TrLRed
# Now the blue ones are early - now just enter loop over number of turns
#   later - for longitudinal filter cooling - some more tracking will be needed

    beam=[]
    xp_beam=[]
    x_beam=[]
    beam=ptsRed+ptsBlue
    beam = np.array(beam)
    x_beam=beam[:,0]
    xp_beam=beam[:,1]
    dtl_beam=beam[:,2]
    emittance[1]=np.sqrt(np.mean(x_beam**2)*np.mean(xp_beam**2)-np.mean(x_beam*xp_beam)**2) 
#emittance_individual_particles[1][:]=(gamPU*x_beam[:10]**2+2*alfPU*(x_beam[:10]*xp_beam[:10])+betPU*(xp_beam[:10]**2))/2
    rms_momentum_spread[1]=np.sqrt(np.mean(dtl_beam**2)-np.mean(dtl_beam)**2)

# Now we can go in loop and even implement transverse cooling or longitudinal time delay cooling (but not filter)
    for turn in range(Nturns):
    #print(f' turn {turn:5d} after {(time.time() - datin):8.2f} s <===')
#   Get ready for the blue particles to be tracked and track them to kicker
     ptsBlue, ptsRed = PartsExchange( ptsBlue, ptsRed,Th )
     ptsBlueprev, TrLBlueprev = ptsBlue, TrLBlue  # Keep longitudinal trace to add filter cooling later
     time2,TrHBlue, TrLBlue = TimeTraces( ptsRedprev, ptsBlue, ptsRed,t_bins,Th )
     TrLDiffBlue = [h - l for h, l in zip(TrLBlue_prev, TrLBlue)]
     cs = sp.interpolate.CubicSpline( time2, 
                  TrHBlue )
     csL=sp.interpolate.CubicSpline( time2, 
                  np.array(TrLDiffBlue) )
    #cs = sp.interpolate.interp1d( DTSamp*np.arange(-NSampext, NSamp + 1 + NSampext)-Th/2, 
     #             TrHBlue[-NSampext:] + TrHBlue[:NSamp + 1 + NSampext] )
    #csL=sp.interpolate.interp1d( DTSamp*np.arange(-NSampext, NSamp + 1 + NSampext)-Th/2, 
     #             TrLBlue[-NSampext:] + TrLBlue[:NSamp + 1 + NSampext] )

    #print(len(TrHBlue))
     ptsBlue = [ [matPUK11*x + matPUK12*xp, matPUK21*x + matPUK22*xp, delt, 
                 Tau + etaPUK*delt*Trev] for x, xp, delt, Tau in ptsBlue ]
#   One day cooling to be applied -  now just dump time traces instead
     ptsBlue=kickH(ptsBlue,cs,gain)
     ptsBlue=kickL(ptsBlue,csL,gainL)
#   track the blue one further to the PU
     ptsBlue = [ [matKPU11*x + matKPU12*xp, matKPU21*x + matKPU22*xp, delt, 
                 Tau + etaKPU*delt*Trev] for x, xp, delt, Tau in ptsBlue ]
     TrHBlue_prev, TrLBlue_prev = TrHBlue, TrLBlue
#   Get ready for the red particles to be tracked and track them to kicker
     ptsRed, ptsBlue = PartsExchange( ptsRed, ptsBlue,Th )
     ptsRedprev, TrLRedprev = ptsRed, TrLRed  # Keep longitudinal trace to add filter cooling later
     time2,TrHRed, TrLRed = TimeTraces( ptsBlueprev, ptsRed, ptsBlue,t_bins,Th )
     TrLDiffRed = [h - l for h, l in zip(TrLRed_prev, TrLRed)]
     cs = sp.interpolate.CubicSpline( time2, 
                  TrHRed )
     csL=sp.interpolate.CubicSpline( time2, 
                  np.array(TrLDiffRed) )
    
     y_cs = cs(time2)
     y_csL = csL(time2)
    #print("Spline Knots:", cs.x)  # Knots where the spline pieces join
    #print("Spline Coefficients:", cs.c)  # Coefficients of the spline pieces
   
     tauPU=[]
     xPU=[]
     beam=ptsRed
     beam = np.array(beam)
     tauPU=beam[:,3]
     xPU=beam[:,0]
    
    
# Plot the spline
     '''
     if turn==Nturns-1:
      plt.figure(figsize=(8, 5))
      plt.plot(time2, y_cs, label="TrHRed Interpolated", linestyle="-")
     #plt.plot(t_smooth, y_csL, label="TrLDiffRed Interpolated", linestyle="--")
      plt.scatter(time2, TrHRed, marker="o", color="blue", label="horizontal signal")
      plt.scatter(tauPU, xPU, zorder=5, color="red", label="position of particles")
     '''
     
     ptsRed = [ [matPUK11*x + matPUK12*xp, matPUK21*x + matPUK22*xp, delt, 
                Tau + etaPUK*delt*Trev] for x, xp, delt, Tau in ptsRed ]
#   One day cooling(s) to be applied - now just dump time traces instead
     ptsRed=kickH(ptsRed,cs,gain)
     ptsRed=kickL(ptsRed,csL,gainL)   #time of flight method longitudinal cooling
#   track the red ones further to the PU, where they will be the late ones
     ptsRed = [ [matKPU11*x + matKPU12*xp, matKPU21*x + matKPU22*xp, delt, 
                Tau + etaKPU*delt*Trev] for x, xp, delt, Tau in ptsRed ] 
     TrHRed_prev, TrLRed_prev = TrHRed, TrLRed
     beam=[]
     xp_beam=[]
     x_beam=[]
     delta_beam=[]
     dlt_final=[]
     tau_final=[]
     beam=ptsRed+ptsBlue
     beam = np.array(beam)
     x_beam=beam[:,0]
     xp_beam=beam[:,1]
     delta_beam=beam[:,2]
     tau_final=beam[:,3]
     dlt_final=beam[:,2]
     counts_final, bins_final = np.histogram(dlt_final, bins=30)
     emittance[turn+2]=np.sqrt(np.mean(x_beam**2)*np.mean(xp_beam**2)-np.mean(x_beam*xp_beam)**2)
     geo_emittance[turn+2]=(gamPU*np.mean(x_beam**2)+2*alfPU*np.mean(x_beam*xp_beam)+betPU*np.mean(xp_beam**2))/2
     rms_momentum_spread[turn+2]=np.sqrt(np.mean(delta_beam**2)-np.mean(delta_beam)**2)
    #emittance_individual_particles[turn+2][:]=(gamPU*x_beam[:10]**2+2*alfPU*(x_beam[:10]*xp_beam[:10])+betPU*(xp_beam[:10]**2))/2

    print(f'The initial emittance is {emittance[0]}{chr(0x03C0)} mm*mrad')
    print(f'The final emittance after cooling is {emittance[-1]}{chr(0x03C0)} mm*mrad')
    print(f'Is emittance reduced? -->  {emittance[-1] < emittance[0]}')
    if emittance[-1] < emittance[0]:
      print(f"Emittance is reduced by {(emittance[0] - emittance[-1]) * 100 / emittance[0]}%")
    print(f'The initial rms_momentum_spread is {rms_momentum_spread[0]} ')
    print(f'The final rms_momentum_spread after cooling is {rms_momentum_spread[-1]} ')
    print(f'Is rms_momentum_spread reduced? -->  {rms_momentum_spread[-1] < rms_momentum_spread[0]}')
    if rms_momentum_spread[-1] < rms_momentum_spread[0]:
      print(f"rms_momentum_spread is reduced by {(rms_momentum_spread[0] - rms_momentum_spread[-1]) * 100 / rms_momentum_spread[0]}%")

    return emittance[-1], rms_momentum_spread[-1]


In [11]:
#3000 particles 20000 turns

from memory_profiler import memory_usage


mem_usage = memory_usage(lambda :Cooling(0.0002,2e-8,3000,20000), interval=0.1)
print(f"Peak memory usage: {max(mem_usage):.2f} MB")

========> 800 1 1.2499999999999998e-10 1e-07 25.600000000000005
The initial emittance is 0.9785205504150565π mm*mrad
The final emittance after cooling is 0.050902226014428925π mm*mrad
Is emittance reduced? -->  True
Emittance is reduced by 94.79804220843008%
The initial rms_momentum_spread is 9.920147303663395e-05 
The final rms_momentum_spread after cooling is 1.4873687125500405e-05 
Is rms_momentum_spread reduced? -->  True
rms_momentum_spread is reduced by 85.00658642436919%
Peak memory usage: 258.30 MB


In [13]:
#8000 particles 20000 turns

from memory_profiler import memory_usage


mem_usage = memory_usage(lambda :Cooling(0.0002,2e-8,8000,20000), interval=0.1)
print(f"Peak memory usage: {max(mem_usage):.2f} MB")

========> 800 1 1.2499999999999998e-10 1e-07 25.600000000000005
The initial emittance is 1.0015662118483675π mm*mrad
The final emittance after cooling is 1837.7873351912053π mm*mrad
Is emittance reduced? -->  False
The initial rms_momentum_spread is 0.00010005445134838862 
The final rms_momentum_spread after cooling is 1.5550493599113113e-05 
Is rms_momentum_spread reduced? -->  True
rms_momentum_spread is reduced by 84.45796924619931%
Peak memory usage: 262.68 MB
